# Overall Workflow
## 1. The Dataset
## 2. Preprocessing Before Modeling
## 3. Hyperparameter Tuning - Bayesian Optimization

# Packages

In [1]:
# Data structures
import numpy as np
import pandas as pd

# Data formats
import csv
import json

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sb

# Statistical testings
from scipy.stats import f
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

# Data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine learning algorithms
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb 
import lightgbm as lgb

# Hyperparameter tuning
from hyperopt import hp
from hyperopt import Trials
from hyperopt import tpe
from timeit import default_timer as timer
from hyperopt import STATUS_OK
from hyperopt import fmin
from bayes_opt import BayesianOptimization

# Model validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

# Improvement
from imblearn.over_sampling import SMOTE

/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


# 1. The Dataset

In [2]:
data = pd.read_csv('banking.csv')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [3]:
# 41188 data points with 21 predictors.
print(data.shape)

(41188, 21)


In [4]:
# Since the purpose is to build a prediction model, I dropped this column from the feature space.
data.drop(columns='duration', inplace=True)
print(data.shape)

(41188, 20)


# 2. Preprocessing Before Modeling
- **2.1 One-hot encoding**
- **2.2 Dummy feature generation**
- **2.3 Improvement: Over-sampling using SMOTE**
- **2.4 Train & test splitting (80%/20%)**
- **2.5 Standardization**
    - Why? Standardization for the regularization in Logistic Regression and SGD classifier purposes.

## 2.1 One-hot encoding

In [5]:
dummy_features = []
binary_features = []
for each in data.columns:
    if data[each].dtype == 'object':
        if len(data[each].unique()) > 2:
            dummy_features.append(each)
        else: 
            binary_features.append(each)

In [6]:
# One-hot encoding for categorical features containing two levels.
le = LabelEncoder()
le_count = 0

for col in binary_features:
    print(col)
    le.fit(data[col])
    data[col] = le.transform(data[col])
            
    # Keep track of how many columns were label encoded
    le_count += 1
            
print('%d columns were label encoded.' % le_count)

contact
1 columns were label encoded.


In [7]:
# Make sure the transformation is correct!
# 0: cellular
# 1: telephone
data['contact'].unique()

array([0, 1])

## 2.2 Dummy feature generation

In [8]:
# Create dummy features.
sum_len = 0
for col in dummy_features:
    sum_len += len(data[col].unique())
    print(col, len(data[col].unique()))
    
print('{} columns will be additionally added.'.format(sum_len - len(dummy_features)))

job 12
marital 4
education 8
default 3
housing 3
loan 3
month 10
day_of_week 5
poutcome 3
42 columns will be additionally added.


In [9]:
# Original dimension
print(data.shape)

# 20 + 42 = 62, correct dummy feature transformation!
data = pd.get_dummies(data, columns = dummy_features)
print(data.shape)

(41188, 20)
(41188, 62)


In [10]:
data.head()

,age,contact,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,44,0,1,999,0,1.4,93.444,-36.1,4.963,5228.1,...,0,0,0,0,1,0,0,0,1,0
1,53,0,1,999,0,-0.1,93.200,-42.0,4.021,5195.8,...,0,0,1,0,0,0,0,0,1,0
2,28,0,3,6,2,-1.7,94.055,-39.8,0.729,4991.6,...,0,0,0,0,1,0,0,0,0,1
3,39,0,2,999,0,-1.8,93.075,-47.1,1.405,5099.1,...,0,0,1,0,0,0,0,0,1,0
4,55,0,1,3,1,-2.9,92.201,-31.4,0.869,5076.2,...,0,0,1,0,0,0,0,0,0,1


## 2.3 Improvement: Over-sampling using SMOTE 

In [11]:
os = SMOTE(sampling_strategy='minority', k_neighbors=10, m_neighbors=20, random_state=0)
os_data_X, os_data_y = os.fit_sample(data[data.columns.difference(['y'])], 
                                     data.y)
os_data_X = pd.DataFrame(data=os_data_X, columns=data.columns.difference(['y']))
os_data_y = pd.DataFrame(data=os_data_y,columns=['y'])

# Check the numbers of over-sampled data
print("length of oversampled data is ", len(os_data_X))
print("Number of no subscription in oversampled data", len(os_data_y[os_data_y['y'] == 0]))
print("Number of subscription", len(os_data_y[os_data_y['y'] == 1]))
print("Proportion of no subscription data in oversampled data is ",
      len(os_data_y[os_data_y['y'] == 0])/len(os_data_y))
print("Proportion of subscription data in oversampled data is ",
      len(os_data_y[os_data_y['y'] == 1])/len(os_data_y))

length of oversampled data is  73096
Number of no subscription in oversampled data 36548
Number of subscription 36548
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


## 2.4 Train & test splitting (80%/20%)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(os_data_X, 
                                                    os_data_y, 
                                                    test_size=0.2, random_state=0) 

In [13]:
# Make sure the splitting is correct!
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(58476, 61) (14620, 61)
(58476, 1) (14620, 1)


## 2.5 Standardization

In [14]:
# Test data are unseen data! 
# Since my purpose is to build a prediction model, I shouldn't contaminate train and test data together. 
std_scaler = StandardScaler().fit(x_train)

train_standardized_data = std_scaler.transform(x_train)
test_standardized_data = std_scaler.transform(x_test)

In [15]:
# Make sure the standardization is correct (mean must be 0 and variance must be 1) 
# Train data
pd.DataFrame(train_standardized_data).describe()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
count,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,...,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04,5.847600e+04
mean,-1.002020e-14,6.748655e-17,-6.869270e-14,-2.301517e-11,3.667442e-15,-3.870341e-15,-3.901273e-16,1.718602e-15,1.675090e-15,-2.863487e-15,...,-1.329894e-15,-4.945292e-16,1.779803e-15,1.507841e-15,1.183280e-11,-3.371684e-15,1.082046e-15,2.154010e-15,-1.583188e-15,-1.971310e-15
std,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,...,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00
min,-1.962015e+00,-5.680033e-01,-1.983291e+00,-2.026413e+00,-6.322546e-01,-5.095551e-01,-5.276203e-01,-5.619754e-01,-5.346215e-01,-5.361716e-01,...,-6.139690e-01,-3.350478e-01,-2.040029e-01,-1.854645e-01,-1.984152e+00,-2.851768e+00,-3.683961e-01,-1.983339e+00,-3.412183e-01,-4.575588e-01
25%,-7.633044e-01,-5.680033e-01,-4.652984e-01,-8.507732e-01,-6.322546e-01,-5.095551e-01,-5.276203e-01,-5.619754e-01,-5.346215e-01,-5.361716e-01,...,-6.139690e-01,-3.350478e-01,-2.040029e-01,-1.854645e-01,-6.875851e-01,3.529286e-01,-3.683961e-01,5.307416e-01,-3.412183e-01,-4.575588e-01
50%,-1.997150e-01,-1.843367e-01,-2.966325e-01,-5.559780e-02,-6.322546e-01,-5.095551e-01,-5.276203e-01,-5.619754e-01,-5.346215e-01,-5.361716e-01,...,-6.139690e-01,-3.350478e-01,-2.040029e-01,-1.854645e-01,6.343143e-01,3.529286e-01,-3.683961e-01,5.307416e-01,-3.412183e-01,-4.575588e-01
75%,6.394753e-01,2.584066e-01,7.153626e-01,8.164442e-01,1.645423e+00,-5.095551e-01,-4.646568e-01,5.189206e-02,-2.971104e-01,-2.754336e-01,...,1.657902e+00,-3.350478e-01,-2.040029e-01,-1.854645e-01,1.061513e+00,3.529286e-01,-3.683961e-01,5.307416e-01,-3.412183e-01,-4.575588e-01
max,4.835427e+00,2.323110e+01,2.495724e+00,2.042060e+00,1.657076e+00,2.238409e+00,2.150412e+00,2.026440e+00,2.141712e+00,2.135584e+00,...,1.657902e+00,3.104917e+00,5.185049e+00,5.793828e+00,1.061513e+00,3.529286e-01,3.026884e+00,5.307416e-01,2.984327e+00,1.018560e+01


# 3. Hyperparameter Tuning - Bayesian Optimization
#### Each model was optimized 100 times to obtain the optimal hyperparameters.
- **3.1 Logistic Regression**
- **3.2 SGD Classifier**
- **3.3 Decision Tree**
- **3.4 Bagging - Decision Tree**
- **3.5 Random Forest**
- **3.6 AdaBoost**
- **3.7 Gradient Boosting**
- **3.8 XGBoost**
- **3.9 LightGBM**

## 3.1 Logistic Regression
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    - penalty: Used to specify the norm used in the penalization.
    - C: Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
    - fit_intercept: Specifies if a constant (a.k.a. bias or intercept) should be added to the decision function.
    - random_state: The seed of the pseudo random number generator to use when shuffling the data.

In [ ]:
# Define the search space (hyperparameter space).
space = {
    'penalty': hp.choice('penalty', ['l2', 'l1']),
    'C': hp.uniform('C', 0.01, 1000.0),
    'fit_intercept':hp.choice('fit_intercept', [True, False]),
    'random_state':0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = LogisticRegression(**hyperparameters)
    cv_results = cross_validate(model, train_standardized_data, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
def bayesian_optimize(n_eval):
    
    # Record searching results.
    trials = Trials()
    
    # Create a csv file to store results.
    of_connection = open(OUT_FILE, 'w')
    writer = csv.writer(of_connection)

    ITERATION = 0
    
    # Write column names in the file.
    headers = ['loss', 'hyperparameters', 'iteration', 'runtime']
    writer.writerow(headers)
    of_connection.close()
    
    
    # Run x evaluations.
    MAX_EVALS = n_eval

    # Start optimization!
    best = fmin(fn = objective, 
                space = space, 
                algo = tpe.suggest, 
                trials = trials,
                max_evals = MAX_EVALS)

    return best

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_LR.csv'

In [ ]:
bayesian_optimize(100)

## 3.2 SGD Classifier
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
    - loss: The loss function to be used.
    - penalty: The penalty (aka regularization term) to be used.
    - alpha: Constant that multiplies the regularization term.
    - l1_ratio: The Elastic Net mixing parameter. l1_ratio=0 corresponds to L2 penalty, and l1_ratio=1 to L1.
    - fit_intercept: Whether the intercept should be estimated or not.

In [ ]:
# Define the search space
space = {
    'loss':'log',
    'penalty': hp.choice('penalty', ['none', 'l2', 'l1', 'elasticnet']),
    'alpha': hp.uniform('alpha', 0.0001, 100.0),
    'l1_ratio': hp.uniform('l1_ratio', 0.0, 1.0),
    'fit_intercept':hp.choice('fit_intercept', [False, True]),
    'random_state':0
}

In [ ]:
# Define the objective (here is to minimize mse)
def objective(hyperparameters):
    
    """Objective function for multiple linear regression hyperparameter optimization."""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = SGDClassifier(**hyperparameters)
    cv_results = cross_validate(model, train_standardized_data, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_SGD.csv'

In [ ]:
bayesian_optimize(100)

## 3.3 Decision Tree
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
    - criterion: The function to measure the quality of a split.
    - max_depth: The maximum depth of the tree.
    - min_samples_split: The minimum number of samples required to split an internal node.
    - min_samples_leaf: The minimum number of samples required to be at a leaf node. (Will produce a conflict since max_depth determines the number of samples at a leaf node too.)
    - random_state

In [ ]:
# Define the search space (hyperparameter space).
space = {
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.randint('max_depth', 58476-1),
    'min_samples_split': hp.uniform('min_samples_split', 0.0, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.0, 0.5),
    'random_state': 0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = DecisionTreeClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_DT.csv'

In [ ]:
bayesian_optimize(100)

## 3.4 Bagging - Decision Tree
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
    - base_estimator: The base estimator to fit on random subsets of the dataset.
    - n_estimators: The number of base estimators in the ensemble.
    - max_samples: The number of samples to draw from X to train each base estimator.
    - bootstrap: Whether samples are drawn with replacement.
    - random_state

In [ ]:
pd.read_csv('Bank_Marketing_DT.csv').sort_values(by='loss').head(5)

In [ ]:
pd.read_csv('Bank_Marketing_DT.csv').sort_values(by='loss').hyperparameters.head(1).values

In [ ]:
# Define the search space (hyperparameter space).

# For bagging, it needs to have a base estimator as a parameter. 
# Thus I chose the top one optimal decision tree and a default decision tree
# (to perform bagging, it's better to have a 'deep' tree).
optimal_dt = DecisionTreeClassifier(criterion='entropy',
                                    max_depth=52983,
                                    min_samples_leaf=0.0005254964145084773,
                                    min_samples_split=0.03703819684650395,
                                    random_state=0)

default_dt = DecisionTreeClassifier(max_depth=15, random_state=0)

space = {
    'base_estimator': hp.choice('base_estimator', [optimal_dt, default_dt]),
    'n_estimators': hp.randint('n_estimators', 500),
    'max_samples': hp.uniform('max_samples', 0.0, 1.0),
    'bootstrap': hp.choice('bootstrap', [False, True]),
    'random_state': 0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = BaggingClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_Bag_DT.csv'

In [ ]:
bayesian_optimize(100)

## 3.5 Random Forest
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    - n_estimators: The number of trees in the forest.
    - criterion: The function to measure the quality of a split.
    - max_depth: The maximum depth of the tree.
    - min_samples_split: The minimum number of samples required to split an internal node.
    - min_samples_leaf: The minimum number of samples required to be at a leaf node.
    - max_features: The number of features to consider when looking for the best split.
    - bootstrap: Whether bootstrap samples are used when building trees.
    - random_state

In [ ]:
space = {
    'n_estimators': hp.randint('n_estimators', 500),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.randint('max_depth', 500),
    'min_samples_split': hp.uniform('min_samples_split', 0.0, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.002, 0.5),
    'max_features': hp.uniform('max_features', 0.0, 1.0),
    'bootstrap': hp.choice('bootstrap', [False, True]),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = RandomForestClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_RF.csv'

In [ ]:
bayesian_optimize(100)

## 3.6 AdaBoost
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
    - base_estimator: The base estimator from which the boosted ensemble is built.
    - n_estimators: The maximum number of estimators at which boosting is terminated.
    - learning_rate: Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators.
    - random_state

In [ ]:
# Typically, boosting requires "stumps" (shallow trees with depth from 4 to 8) to improve
# the model performance.
base_1 = DecisionTreeClassifier(max_depth=4,random_state=0)
base_2 = DecisionTreeClassifier(max_depth=5,random_state=0)
base_3 = DecisionTreeClassifier(max_depth=6,random_state=0)
base_4 = DecisionTreeClassifier(max_depth=7,random_state=0)
base_5 = DecisionTreeClassifier(max_depth=8,random_state=0)

In [ ]:
space = {
    'base_estimator': hp.choice('base_estimator', [base_1,base_2,base_3,base_4,base_5]),
    'n_estimators': hp.randint('n_estimators', 500),
    'learning_rate': hp.uniform('learning_rate', 0.0, 1.0),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = AdaBoostClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_AdaBoost.csv'

In [ ]:
bayesian_optimize(100)

## 3.7 Gradient Boosting
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
    - n_estimators: The number of boosting stages to perform.
    - max_depth: maximum depth of the individual regression estimators.
    - subsample: The fraction of samples to be used for fitting the individual base learners.
    - max_features: The number of features to consider when looking for the best split.
    - learning_rate: learning rate shrinks the contribution of each tree by learning_rate. There is a trade-off between learning_rate and n_estimators.
    - random_state

In [ ]:
space = {
    'n_estimators': hp.randint('n_estimators', 500),
    'max_depth': hp.uniform('max_depth', 0, 8),
    'subsample': hp.uniform('subsample', 0.0, 1.0),
    'max_features': hp.uniform('max_features', 0.0, 1.0),
    'learning_rate': hp.uniform('learning_rate', 0.0, 1.0),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = GradientBoostingClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_GradientBoost.csv'

In [ ]:
bayesian_optimize(100)

## 3.8 XGBoost
- Reference: https://xgboost.readthedocs.io/en/latest/parameter.html
    - Parameters for Tree Booster
        - eta: Step size shrinkage used in update to prevents overfitting.
        - max_depth: Maximum depth of a tree.
        - subsample: Subsample ratio of the training instances.
        - colsample_bytree: Subsample ratio of columns when constructing each tree.
        - colsample_bylevel: Subsample ratio of columns for each level.
        - colsample_bynode: Subsample ratio of columns for each node (split).
        - lambda: L2 regularization term on weights.
        - alpha: L1 regularization term on weights.
    - Learning Task Parameters
        - objective: binary:logistic (logistic regression for binary classification, output probability).
        - eval_metric: auc (Area under the curve).
    - Command Line Parameters
        - num_round: The number of rounds for boosting.
        - task: train (training using data).

In [ ]:
space = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': hp.uniform('eta', 0.0, 1.0),
    'max_depth': hp.randint('max_depth', 8),
    'subsample': hp.uniform('subsample', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.0, 1.0),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.0, 1.0),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.0, 1.0),
    'lambda': hp.uniform('lambda', 0.0, 1.0),
    'alpha': hp.uniform('alpha', 0.0, 1.0),
    'num_round': hp.randint('num_round', 500),
    'task': 'train'
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    cv = xgb.cv(hyperparameters, xgb_train, num_boost_round=hyperparameters['num_round'], 
                nfold=5, early_stopping_rounds=10, verbose_eval=0)
    #print(cv)
    
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(cv['test-auc-mean'][cv.shape[0]-1])
    
    # Update the number of boosting rounds once the early stopping is finished.
    hyperparameters['num_round'] = cv.shape[0]

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
xgb_train = xgb.DMatrix(x_train, label=y_train)

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Bank_Marketing_XGB.csv'

In [ ]:
bayesian_optimize(100)

## 3.9 LightGBM
- Reference:
    - Core parameters
        - task: default = train.
        - objective: binary, binary log loss classification (or logistic regression).
        - boosting: default = gbdt (traditional Gradient Boosting Decision Tree).
        - data: path of training data, LightGBM will train from this data.
        - num_iterations: number of boosting iterations.
        - learning_rate: shrinkage rate.
        - num_leaves: max number of leaves in one tree
        - seed: this seed is used to generate other seeds.
    - Learning Control Parameters
        - max_depth: limit the max depth for tree model. 
        - bagging_fraction: randomly select part of data without resampling.
        - bagging_freq: 0 means disable bagging; k means perform bagging at every k iteration.
        - feature_fraction: randomly select part of features on each iteration.
        - early_stopping_round: will stop training if one metric of one validation data doesn't improve in last early_stopping_round rounds.
        - lambda_l1: L1 regularization.
        - lambda_l2: L2 regularization.
    - Metric Parameters
        - metric: metric(s) to be evaluated on the evaluation set(s).

In [ ]:
def bayes_parameter_opt_lgb(X, y, file_path, init_round=15, opt_round=25, n_folds=5, 
                            random_seed=0, output_process=False):
    # Prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    
    # Hyperparameters
    def lgb_eval(num_iterations, learning_rate, max_depth,
                 bagging_fraction, feature_fraction, 
                 lambda_l1, lambda_l2):
        # Predefined parameters
        params = {
            "objective":"binary", "seed":0, "metric":"auc", "bagging_freq":1
        }
        # Hyperparameters
        params['num_iterations'] = int(round(num_iterations))
        params['learning_rate'] = learning_rate
        params['max_depth'] = int(round(max_depth))
        params['bagging_fraction'] = bagging_fraction
        params['feature_fraction'] = feature_fraction
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        
        # 5-fold cross validations
        cv_result = lgb.cv(params, train_set=train_data, 
                           num_boost_round=num_iterations,
                           nfold=n_folds, 
                           early_stopping_rounds=10,
                           seed=random_seed, 
                           verbose_eval=200, 
                           stratified=False)
        
        # Update the number of boosting rounds once the early stopping is finished.
        params['num_iterations'] = len(cv_result['auc-mean'])
        
        return cv_result['auc-mean'][-1]
    
    
    # Hyperparameter range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_iterations': (100, 500),
                                            'learning_rate': (0.1, 1.0),
                                            'max_depth': (1, 8),
                                            'bagging_fraction': (0.1, 1.0),
                                            'feature_fraction': (0.1, 1.0),
                                            'lambda_l1': (0.0, 1.0),
                                            'lambda_l2': (0.0, 1.0)}, 
                                 random_state=0)
    
    # Start optimizing (maximize the auc score)
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

    # Output optimization process
    if output_process==True: 
        # Write searching results to a csv file ('a' means append)
        of_connection = open(file_path, 'a')
        writer = csv.writer(of_connection)
        headers = ['loss', 'hyperparameters', 'iteration']
        writer.writerow(headers)
        for i in range(len(lgbBO.res)):
            writer.writerow([
                lgbBO.res[i]['target'], lgbBO.res[i]['params'], i])
        of_connection.close()
        
    # Return the searching result
    return lgbBO.res

In [74]:
# In total, performed 100 times of optimization.
opt_params = bayes_parameter_opt_lgb(X=x_train, y=y_train, 
                                     file_path='Bank_Marketing_LightGBM.csv',
                                     init_round=10, opt_round=90, n_folds=5, 
                                     random_seed=0, output_process=True)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | num_it... |
-------------------------------------------------------------------------------------------------------------


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  1        |  0.9736   |  0.5939   |  0.7437   |  0.6028   |  0.5449   |  0.4813   |  5.521    |  275.0    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  2        |  0.9736   |  0.9026   |  0.9673   |  0.3834   |  0.7917   |  0.576    |  4.976    |  470.2    |


...

|  98       |  0.9752   |  0.9918   |  0.3085   |  0.9125   |  0.04186  |  0.1069   |  7.644    |  385.4    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  99       |  0.9739   |  0.1      |  0.1      |  0.0      |  0.0      |  0.1      |  4.709    |  400.5    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  100      |  0.9729   |  0.8829   |  0.1073   |  0.05346  |  0.8142   |  0.9542   |  7.746    |  101.8    |
